## Tri-training


In [11]:
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType,StringType,DoubleType
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
import sklearn
import pandas as pd 
 
from pyspark.sql.functions import col



# The implementation of Tri-training algorithm
This part of the code is the implementation of Tri-training algorithm. 

In [13]:
class TriTraining:
    def __init__(self, classifier):

            
              self.classifiers = [classifier[0],classifier[1],classifier[2]] 
              self.models=[[]]*3
              

    def fit(self, train):
        #split to label and unlabel set
        L_train, U_train = train.randomSplit([1-unlabel_ratio, unlabel_ratio], seed=rand_seed)

        U_X = U_train.drop("ClassIndex")
        
        L_X, L_Y ,L_Full= [[]] * 3, [[]] * 3 ,[[]] * 3  
 
        L_train_x= L_train.select("features")
        L_train_y= L_train.select("ClassIndex")

        L_np=L_train.toPandas();

        # boostrap
        for i in range(3):
            sample = sklearn.utils.resample(L_np)
            L_Full[i]=spark.createDataFrame(sample) 
            L_Full[i].cache()
            self.models[i]=self.classifiers[i].fit(L_Full[i])
  
                          
        e_previous = [0.5] * 3 #default upper bound error
        l_previous = [0] * 3
        e = [0] * 3 
        update = [False] * 3
  
        Li_X, Li_y = [[]] * 3, [[]] * 3  

        improve = True
        self.iter = 0

        while improve:
                  self.iter += 1  

                  for i in range(3):
                      

                      j, k = np.delete(np.array([0, 1, 2]), i)
                      update[i] = False
                    
                      # obtain the classification error rate of the hypothesis from the combination of the other two classifier
                      e[i] = self.measure_error(L_train_x, L_train_y,L_train, j, k) 
                      
                      
                      if e[i] < e_previous[i]:

                          Unlabel_j = self.models[j].transform(U_X)
                          Unlabel_k = self.models[k].transform(U_X)
                          # Join two DataFrames containing the prediction results of hj and hk and find the sample that hj and hk agree
                          #on the labeling of the example by filter()
                          Unlabel_j=Unlabel_j.join(Unlabel_k, (Unlabel_j["features"] == Unlabel_k["features"]),"left")\
                                             .withColumn("sub", when(Unlabel_k.prediction == Unlabel_j.prediction, 0).otherwise(1))\
                                             .drop(Unlabel_k["features"])\
                                             .drop(Unlabel_k["probability"])\
                                             .drop(Unlabel_k["rawPrediction"])\
                                             .drop(Unlabel_k["prediction"])

                          
                          Unlabel_j_equal= Unlabel_j.filter(Unlabel_j.sub == 0)
                          Li_X[i]=Unlabel_j_equal
                          Unlabel_j_y= Unlabel_j_equal.select("prediction")
                          Li_y[i]=Unlabel_j_y
                          
                          
                          
                          if l_previous[i] == 0:  # no updated before
                              l_previous[i] = int(e[i] / (e_previous[i] - e[i]) + 1)
                          if l_previous[i] < Li_X[i].count():
                              if e[i] * Li_X[i].count() < e_previous[i] * l_previous[i]:
                                    update[i] = True
                              elif l_previous[i] > e[i] / (e_previous[i] - e[i]):  #after subsampling the size of current Li_X will be still larger than l_previous[i]
                                    x = int(e_previous[i] * l_previous[i] / e[i] - 1)
                                    y = Li_y[i].count()    
                                    Li_X[i]= Li_X[i].sample(False,x/y,42) #random select x/y sample in unlabel dataset, 
                                    Li_y[i] = Li_X[i].select("prediction")
                                    update[i] = True
                          
                          
                  for i in range(3):
                          if update[i]:
                              Li_X[i]=Li_X[i].drop("probability","rawPrediction","sub");
                              Li_X[i]=Li_X[i].withColumnRenamed("prediction","ClassIndex")
                              Li_X[i] = Li_X[i].select(["features","ClassIndex"])
                              L_Union=L_train.union(Li_X[i])
                              L_Union.cache()  
                              self.models[i]=self.classifiers[i].fit(L_Union) #retrain the model by the augumented data
                              e_previous[i] = e[i]
                              l_previous[i] = Li_y[i].count()
                              L_Union.unpersist()

                  Unlabel_j_equal.unpersist()
                  Unlabel_j_y.unpersist()   
                  Li_X[i].unpersist()
                  Li_y[i].unpersist()    


                  if update == [False] * 3:
                          improve = False  # if no classifier was updated, no improvement and break the loop

        return self.models;


# The classification error rate of the hypothesis is approximated by dividing the number of labeled examples on which both model j and k make
# incorrect classification by the number of labeled examples on which the classification made by j is the same as that made by k.
    def measure_error(self, X, y, L_train, j, k):
       
        j_pred = self.models[j].transform(X)
        k_pred = self.models[k].transform(X)

        k_pred = k_pred.select("features","prediction")
        j_pred = j_pred.select("features","prediction")
        L= L_train.join(j_pred,(j_pred["features"] == L_train["features"]),'left')\
            .withColumn("j_right_prediction", when(j_pred.prediction == L_train.ClassIndex, 1)\
            .otherwise(0))\
            .drop(j_pred.features)

        L= L.join(k_pred,(k_pred["features"] == L["features"]),'left')\
            .withColumn("j_k_wrong_prediction",when ((j_pred.prediction==k_pred.prediction)\
             & (L.j_right_prediction==0) , 1).otherwise(0))\
            .withColumn("j_k_same_prediction",when (j_pred.prediction==k_pred.prediction, 1)\
            .otherwise(0))

        j_index=L.filter(L.j_k_same_prediction== 1)
        index = j_index.count()

        j_wrong_index=L.filter(L.j_k_wrong_prediction== 1)
        wrong_index = j_wrong_index.count()
     
        
        L.unpersist()
        j_pred.unpersist()
        k_pred.unpersist()
        

        return wrong_index / index

# Voting mechanism and ensemble learning: if there are more than one classifier with the same prediction result, the predicted value should be this result.
    def predict(self, X):
            
            
              pred_0 =  self.models[0].transform(X)
              pred_1 =  self.models[1].transform(X)
              pred_2 =  self.models[2].transform(X)
              # subtract to obtain whether the two model have same prediction results
              pred_1= pred_1.join(pred_2,(pred_1["features"] == pred_2["features"]),"left").withColumn("sub12", pred_1.prediction -pred_2.prediction)\
                            .drop(pred_2["features"])\
                            .drop(pred_2["probability"])\
                            .drop(pred_2["rawPrediction"])\
                            .drop(pred_2["prediction"])
              pred_1= pred_1.withColumnRenamed("prediction","pred_prediction")
              
              pred_0= pred_0.join(pred_1,(pred_0["features"] == pred_1["features"]),"left")\
                            .drop(pred_1["features"])\
                            .drop(pred_1["probability"])\
                            .drop(pred_1["rawPrediction"])\
              # If the other two model agree on the results, then set this result as the final prediction results
              pred_0= pred_0.withColumn("prediction", when(pred_0.sub12==0,pred_0.pred_prediction).otherwise(pred_0.prediction))

              return pred_0

# Initialization
 This part of the code initializes spark session, dataframe and split the data

In [15]:

import numpy as np
from numpy import allclose
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.linalg import Vectors
from  pyspark.ml.classification import RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import expr

from pyspark.sql.functions import when


spark = SparkSession.builder.master("local[*]") \
                    .appName('Posture') \
                    .getOrCreate()

path = "new_feature.csv"

schema = StructType()\
      .add("Class",DoubleType(),True) \
      .add("X0",DoubleType(),True) \
      .add("Y0",DoubleType(),True) \
      .add("Z0",DoubleType(),True) \
      .add("X1",DoubleType(),True) \
      .add("Y1",DoubleType(),True) \
      .add("Z1",DoubleType(),True) \
      .add("X2",DoubleType(),True) \
      .add("Y2",DoubleType(),True) \
      .add("Z2",DoubleType(),True) \
      .add("X3",DoubleType(),True) \
      .add("Y3",DoubleType(),True) \
      .add("Z3",DoubleType(),True) \

df = spark.read.option("header",True).schema(schema).csv(path)


assembler = VectorAssembler(inputCols=df.drop('Class').columns,outputCol="features")
indexer = StringIndexer(inputCol = 'Class', outputCol = 'ClassIndex')
df=assembler.transform(df)
df = indexer.fit(df).transform(df)

df=df.select('features', 'ClassIndex')
num=df.rdd.getNumPartitions()
df = df.repartition(num*3) 

unlabel_ratio=0.95
label_ratio=0.05
test_ratio=0.10
rand_seed=np.random.randint(99999)


train, test = df.randomSplit([1-test_ratio, test_ratio], seed=rand_seed)
train.cache()
test.cache()


DataFrame[features: vector, ClassIndex: double]

# Train

This part of the code initialize and train our model

In [16]:

n=4
m=12


triTraining=TriTraining([RandomForestClassifier(numTrees=n,maxDepth=m,labelCol='ClassIndex',featuresCol='features',bootstrap=False),
                         RandomForestClassifier(numTrees=n,maxDepth=m,labelCol='ClassIndex',featuresCol='features',bootstrap=False),
                         RandomForestClassifier(numTrees=n,maxDepth=m,labelCol='ClassIndex',featuresCol='features',bootstrap=False)])

triTraining.fit(train)



+--------------------+----------+
|            features|ClassIndex|
+--------------------+----------+
|[-51.253731022324...|       1.0|
|[-28.911238153551...|       2.0|
|[-28.062389062568...|       2.0|
|[-26.715981589456...|       2.0|
|[-24.423199379777...|       2.0|
|[-21.962989974915...|       0.0|
|[-21.379566046016...|       1.0|
|[-21.372414890111...|       1.0|
|[-21.334013756175...|       1.0|
|[-20.266315168748...|       2.0|
|[-19.016337895788...|       2.0|
|[-18.859860263954...|       4.0|
|[-18.664830265849...|       1.0|
|[-17.262166457941...|       2.0|
|[-16.928109152465...|       2.0|
|[-16.650486451779...|       2.0|
|[-16.580251840588...|       1.0|
|[-16.468579851620...|       3.0|
|[-14.909475968550...|       0.0|
|[-14.667052706879...|       0.0|
+--------------------+----------+
only showing top 20 rows



[RandomForestClassificationModel: uid=RandomForestClassifier_e28523cc951e, numTrees=4, numClasses=5, numFeatures=12,
 RandomForestClassificationModel: uid=RandomForestClassifier_a350297f4c6e, numTrees=4, numClasses=5, numFeatures=12,
 RandomForestClassificationModel: uid=RandomForestClassifier_b54ce1558543, numTrees=4, numClasses=5, numFeatures=12]

# Test
This part of the code tests our model on the test data set

In [17]:
testdata=test.select('features')
testlabel=test.select('ClassIndex')
predict=triTraining.predict(testdata)




+----------+-----+---------------+
|prediction|sub12|pred_prediction|
+----------+-----+---------------+
|       2.0|  0.0|            2.0|
|       1.0| -1.0|            1.0|
|       2.0|  1.0|            2.0|
|       0.0|  0.0|            0.0|
|       3.0|  0.0|            3.0|
|       1.0|  0.0|            1.0|
|       1.0|  0.0|            1.0|
|       2.0|  0.0|            2.0|
|       2.0|  0.0|            2.0|
|       1.0|  0.0|            1.0|
|       0.0| -4.0|            0.0|
|       3.0|  0.0|            3.0|
|       4.0|  0.0|            4.0|
|       2.0|  0.0|            2.0|
|       3.0|  0.0|            3.0|
|       2.0|  0.0|            2.0|
|       1.0|  0.0|            1.0|
|       4.0|  0.0|            4.0|
|       0.0| -4.0|            0.0|
|       0.0|  0.0|            0.0|
+----------+-----+---------------+
only showing top 20 rows



# Evaluate
This part of the code evaluates the performance of the model on the test data set

In [22]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
test1=test.join(predict,(test["features"] == predict["features"]),"left").withColumn("outcome",when(test.ClassIndex == predict.prediction, 1).otherwise(0))


from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_acc = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="ClassIndex",metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="ClassIndex",metricName="f1")
evaluator_pre = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="ClassIndex",metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="ClassIndex", metricName="weightedRecall")

print('pyspark accuracy: %.6f' %evaluator_acc.evaluate(test1))
print('pyspark f1-score: %.6f' %evaluator_f1.evaluate(test1))
print('pyspark precision: %.6f' %evaluator_pre.evaluate(test1))
print('pyspark recall: %.6f' %evaluator_recall.evaluate(test1))

pyspark accuracy: 0.811398
pyspark f1-score: 0.810928
pyspark precision: 0.813446
pyspark recall: 0.811398


# Stop
This part of the code stops the spark session

In [23]:

spark.stop()